# Question 1

In [1]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd

In [2]:
# Create the optimization model
model = gb.Model("Question 1: Canola Oil")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [3]:
# Read costs from CSV files
direct_shipping_costs_df = pd.read_csv(r"C:\Users\gabri\Downloads\Cost_Production_to_Refinement.csv")
shipping_to_transsipment_costs_df = pd.read_csv(r"C:\Users\gabri\Downloads\Cost_Production_to_Transshipment.csv")
transsipment_to_refinement_costs_df = pd.read_csv(r"C:\Users\gabri\Downloads\Cost_Transshipment_to_Refinement.csv")

In [4]:
# Extract the "Cost" column
direct_shipping_costs = direct_shipping_costs_df['Cost'].values.reshape(25, -1)
shipping_to_transsipment_costs = shipping_to_transsipment_costs_df['Cost'].values.reshape(15, -1)
transsipment_to_refinement_costs = transsipment_to_refinement_costs_df['Cost'].values.reshape(2, -1)

In [5]:
direct_shipping_costs

array([[4.25273277, 4.56772552, 4.69648446, 2.67874096, 4.27245069],
       [2.95047222, 2.63661931, 2.85346871, 5.48002613, 3.03749539],
       [4.02568313, 5.40779944, 3.38094894, 5.44123713, 4.40886217],
       [3.46114107, 5.34694389, 4.99366311, 5.96818684, 4.83600407],
       [5.94681429, 4.74917249, 3.72614981, 3.58915858, 4.93741459],
       [3.46939366, 2.75501443, 5.60123055, 4.49910112, 4.22405746],
       [4.7290315 , 4.30193766, 3.67740309, 4.15881121, 4.15956806],
       [5.74380976, 5.57158666, 5.09534814, 5.34986497, 2.62562966],
       [4.25622982, 5.56865574, 4.13714572, 4.90410382, 3.05688139],
       [4.29664067, 5.75680178, 4.87474585, 4.10968275, 4.66040916],
       [4.02164417, 5.38745866, 5.4089004 , 5.18510303, 3.47259321],
       [3.28957805, 3.58831517, 3.62182771, 4.84919596, 4.89636402],
       [2.9072415 , 4.15142098, 4.60651418, 5.22233913, 3.76568815],
       [5.80493436, 3.58642269, 4.32677984, 5.63769746, 3.39212168],
       [5.87454415, 3.572913  , 3.

In [6]:
shipping_to_transsipment_costs

array([[2.37882586, 0.86384215],
       [1.66698156, 2.11948817],
       [2.17488041, 0.94818376],
       [2.67741553, 0.5924201 ],
       [2.82024783, 0.97232892],
       [2.85389005, 2.6943427 ],
       [1.40702817, 1.42832532],
       [2.48499907, 0.95347666],
       [1.78637201, 0.63669962],
       [0.68852771, 2.25970328],
       [2.40128579, 1.80010665],
       [0.56435019, 2.12484244],
       [1.98606958, 2.34717953],
       [1.63996858, 1.61928103],
       [0.64904663, 1.96881166]])

In [7]:
transsipment_to_refinement_costs

array([[1.57232851, 3.46547396, 2.24406225, 3.77383915, 3.2626518 ],
       [1.90042452, 1.6131232 , 1.70723241, 2.70539568, 2.25742842]])

In [8]:
# Create the a single class of decision variables where
# From = {Ca,US,M,C,F} and To = {R1,R2,R3,R4,R5}.
x = model.addVars(25, 5, lb=0, vtype=GRB.CONTINUOUS, name="Direct Shipping")
# From = {I,U,G} and To = {Italy,Greece}.
y = model.addVars(15, 2, lb=0, vtype=GRB.CONTINUOUS, name="Ship to Transshipment")
# From = {Italy,Greece} and To = {R1,R2,R3,R4,R5}.
z = model.addVars(2, 5, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment to Refinement")

In [9]:
# The objective function
direct_objective = gb.quicksum(direct_shipping_costs[i][j]*x[i,j] for i in range(25) for j in range(5))
trans_objective = gb.quicksum(shipping_to_transsipment_costs[i][j]*y[i,j] for i in range(15) for j in range(2))
trans_to_refinement_objective = gb.quicksum(transsipment_to_refinement_costs[i][j]*z[i,j] for i in range(2) for j in range(5))
model.setObjective(direct_objective + trans_objective + trans_to_refinement_objective, GRB.MINIMIZE)

In [10]:
# Read capacity and demand from CSV files
direct_shipping_supply_capacity_df = pd.read_csv(r"C:\Users\gabri\Downloads\Capacity_for_Direct_Production_Facilities.csv")
transshipment_supply_capacity_df = pd.read_csv(r"C:\Users\gabri\Downloads\Capacity_for_Transship_Distribution_Centers.csv")
shipping_to_transshipment_supply_capacity_df = pd.read_csv(r"C:\Users\gabri\Downloads\Capacity_for_Transship_Production_Facilities.csv")
refinement_demand_df = pd.read_csv(r"C:\Users\gabri\Downloads\Refinement_Demand.csv")

In [11]:
# Extract the "Capacity" and "Demand" column
direct_shipping_supply_capacity = direct_shipping_supply_capacity_df['Capacity'].values.reshape(25, -1)
transshipment_supply_capacity = transshipment_supply_capacity_df['Capacity'].values.reshape(2, -1)
shipping_to_transshipment_supply_capacity = shipping_to_transshipment_supply_capacity_df['Capacity'].values.reshape(15, -1)
refinement_demand = refinement_demand_df['Demand'].values.reshape(5, -1)

In [12]:
direct_shipping_supply_capacity

array([[462],
       [103],
       [460],
       [325],
       [227],
       [217],
       [205],
       [521],
       [548],
       [191],
       [361],
       [411],
       [104],
       [155],
       [285],
       [109],
       [422],
       [438],
       [501],
       [139],
       [462],
       [504],
       [106],
       [132],
       [298]], dtype=int64)

In [13]:
transshipment_supply_capacity

array([[1317],
       [1453]], dtype=int64)

In [14]:
shipping_to_transshipment_supply_capacity

array([[374],
       [444],
       [395],
       [245],
       [378],
       [408],
       [435],
       [175],
       [415],
       [503],
       [184],
       [297],
       [450],
       [169],
       [365]], dtype=int64)

In [15]:
refinement_demand

array([[1537],
       [1748],
       [1940],
       [1838],
       [1665]], dtype=int64)

In [16]:
# Add the supply constraints from source nodes for direct shipping
for i in range(len(direct_shipping_supply_capacity)):
    model.addConstr(gb.quicksum(x[i, j] for j in range(5)) <= direct_shipping_supply_capacity[i], name=f"Direct Supply Constraint {i + 1}")

In [17]:
# Add the supply constraints from source nodes for transshipment shipping
for i in range(len(shipping_to_transshipment_supply_capacity)):
    model.addConstr(gb.quicksum(y[i, j] for j in range(2)) <= shipping_to_transshipment_supply_capacity[i], name=f"Transshipment Supply Constraint {i + 1}")

In [18]:
# Add the supply constraints from transshipment nodes
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) <= transshipment_supply_capacity[0], name="Transship Capacity 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) <= transshipment_supply_capacity[1], name="Transship Capacity 2")

<gurobi.Constr *Awaiting Model Update*>

In [19]:
# Add the flow balance constrainits
model.addConstr(gb.quicksum(y[i,0] for i in range(15)) == gb.quicksum(z[0,k] for k in range(5)), name="Flow Balance 1")
model.addConstr(gb.quicksum(y[i,1] for i in range(15)) == gb.quicksum(z[1,k] for k in range(5)), name="Flow Balance 2")

<gurobi.Constr *Awaiting Model Update*>

In [20]:
# Add the demand constraints
for k in range(len(refinement_demand)):  # Iterate over refinement nodes R1 to R5
    model.addConstr(gb.quicksum(x[i, k] for i in range(25)) + gb.quicksum(z[j, k] for j in range(2)) == refinement_demand[k], name=f"Refinement Demand Constraint {k + 1}")


In [21]:
# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 49 rows, 165 columns and 360 nonzeros
Model fingerprint: 0x7f8fadb4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve time: 0.01s
Presolved: 49 rows, 165 columns, 360 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   8.266000e+03   0.000000e+00      0s
      36    2.4188585e+04   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.418858517e+04


In [22]:
# The status of the model (Optimization Status Codes)
print("Model Status: ", model.status)

Model Status:  2


In [23]:
# Number of variables in the model
print("Number of Decision Variables: ", model.numVars)

Number of Decision Variables:  165


In [24]:
# Value of the objective function
print("Total Transportation cost: ", round(model.objVal, 2))

Total Transportation cost:  24188.59


In [25]:
# Print the decision variables
print(model.printAttr('X'))


    Variable            X 
-------------------------
Direct Shipping[0,3]          462 
Direct Shipping[1,1]          103 
Direct Shipping[2,2]          460 
Direct Shipping[4,3]           86 
Direct Shipping[5,1]          217 
Direct Shipping[7,4]          521 
Direct Shipping[8,4]          548 
Direct Shipping[10,4]          354 
Direct Shipping[11,0]            7 
Direct Shipping[11,2]          404 
Direct Shipping[12,0]          104 
Direct Shipping[13,4]          155 
Direct Shipping[14,3]          285 
Direct Shipping[15,0]          109 
Direct Shipping[17,1]          351 
Direct Shipping[17,4]           87 
Direct Shipping[18,3]          501 
Direct Shipping[20,2]          462 
Direct Shipping[21,3]          504 
Direct Shipping[22,1]          106 
Direct Shipping[23,2]          132 
Ship to Transshipment[0,1]          374 
Ship to Transshipment[2,1]          395 
Ship to Transshipment[3,1]          245 
Ship to Transshipment[6,0]          152 
Ship to Transshipment[7,1]       

# Question 2

In [26]:
# Create the optimization model
question_2_model = gb.Model("Question 2: Sunnyshore Bay")

In [27]:
# Create the three classes of decision variables where each Python
# variable represents a different number of Gurobi decision variables
B = question_2_model.addVars(6, lb=0, vtype=GRB.CONTINUOUS, name="Borrow")
w = question_2_model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Wealth")

In [28]:
# The objective function
question_2_model.setObjective(w[3], GRB.MAXIMIZE)

In [29]:
# Add the balance constraints
question_2_model.addConstr(w[0] == 140000 + 180000 - 300000 + B[0] + B[1] + B[2], "May Balance Constraint")
question_2_model.addConstr(w[1] == w[0] + 260000 - 400000 + B[3] + B[4] - 1.0175*B[2], "June Balance Constraint")
question_2_model.addConstr(w[2] == w[1] + 420000 - 350000 + B[5] - 1.0225*B[1] - 1.0175*B[4], "July Balance Constraint")
question_2_model.addConstr(w[3] == w[2] + 580000 - 200000 - 1.0275*B[0] - 1.0225*B[3] - 1.0175*B[5], "August Balance Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [30]:
# Add the cash flow constraints
question_2_model.addConstr(w[0] >= 25000, "May Cash Flow Constraint")
question_2_model.addConstr(w[1] >= 20000, "June Cash Flow Constraint")
question_2_model.addConstr(w[2] >= 35000, "July Cash Flow Constraint")
question_2_model.addConstr(w[3] >= 18000, "August Cash Flow Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [31]:
# Add the borrowing constraints
question_2_model.addConstr((B[0] + B[1] + B[2]) <= 250000, "May Borrowing Constraint")
question_2_model.addConstr((B[3] + B[4]) <= 150000, "June Borrowing Constraint")
question_2_model.addConstr(B[5] <= 350000, "July Borrowing Constraint")

<gurobi.Constr *Awaiting Model Update*>

In [32]:
# Ratio constraint
question_2_model.addConstr((0.65*(w[0] + w[1])) <= w[2], name="Ratio constraint")

<gurobi.Constr *Awaiting Model Update*>

In [33]:
# Optimally solve the problem
question_2_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 12 rows, 10 columns and 32 nonzeros
Model fingerprint: 0xcc017ff0
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 7 rows, 10 columns, 27 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   1.000000e+30   8.000000e+00      0s
       6    3.2709527e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.270952703e+05


In [34]:
# The status of the model (Optimization Status Codes)
print("Model Status: ", question_2_model.status)

Model Status:  2


In [35]:
# Number of variables in the model
print("Number of Decision Variables: ", question_2_model.numVars)

Number of Decision Variables:  10


In [36]:
# Value of the objective function
print("Total Amount of Money: ", round(question_2_model.objVal, 2))

Total Amount of Money:  327095.27


In [37]:
# Print the decision variables
print(question_2_model.printAttr('X'))


    Variable            X 
-------------------------
   Borrow[0]         5000 
   Borrow[3]      80945.9 
   Borrow[4]      54054.1 
   Wealth[0]        25000 
   Wealth[1]        20000 
   Wealth[2]        35000 
   Wealth[3]       327095 
None
